In [1]:
import pandas as pd
from micromort.data_stores.mysql import db, cursor
from multiprocessing.dummy import Pool as ThreadPool
import traceback
import json
import io
from urlparse import urlparse

In [2]:
tweets_urls = pd.read_csv("../../data/tweetid_url_map.csv", delimiter="\t", header=None, names= ["tweet_id", "url"])
tweets_urls = tweets_urls.drop_duplicates()

In [3]:
tweets_urls.shape, len(tweets_urls.url.unique())

((4460027, 2), 359920)

In [4]:
tweets_urls.head()

,tweet_id,url
0,995956477167898625,https://www.theguardian.com/environment/2018/m...
1,995956478854000641,http://www.bbc.com/news/world-asia-44105279
2,995956485929734145,https://www.nytimes.com/2018/05/13/nyregion/ma...
3,995956486735097857,https://www.theguardian.com/commentisfree/2018...
5,995956487296991232,https://tdy.sg/2GfESie


In [5]:
cursor.execute('Select tweet_id, state, country from newsTweets_location')
location_tweet_data = cursor.fetchall()

In [6]:
len(location_tweet_data)

2149252

In [7]:
location_tweet_data[:10]

((995956476555513856L, '', 'United States'),
 (995956477167898625L, '', 'Netherlands'),
 (995956478144954368L, '', 'India'),
 (995956479059361792L, 'Jharkhand', 'India'),
 (995956478854000641L, 'England', 'United Kingdom'),
 (995956485929734145L, 'Nairobi', 'Kenya'),
 (995956487913549824L, 'Selangor', 'Malaysia'),
 (995956488739803136L, 'Madhya Pradesh', 'India'),
 (995956489276809216L, 'England', 'United Kingdom'),
 (995956490656714752L, '', 'United States'))

### Create a dict of tweet_id: url

In [8]:
tweet_urls_arr = tweets_urls.groupby('tweet_id').agg(lambda x: x.tolist())

In [9]:
dic = {}
count=0
for i, r in tweet_urls_arr.iterrows():
    count=count+1
    dic[r.name] = r["url"]
    if count%20000 == 0:
        print count

20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000
820000
840000
860000
880000
900000
920000
940000
960000
980000
1000000
1020000
1040000
1060000
1080000
1100000
1120000
1140000
1160000
1180000
1200000
1220000
1240000
1260000
1280000
1300000
1320000
1340000
1360000
1380000
1400000
1420000
1440000
1460000
1480000
1500000
1520000
1540000
1560000
1580000
1600000
1620000
1640000
1660000
1680000
1700000
1720000
1740000
1760000
1780000
1800000
1820000
1840000
1860000
1880000
1900000
1920000
1940000
1960000
1980000
2000000
2020000
2040000
2060000
2080000
2100000
2120000
2140000
2160000
2180000
2200000
2220000
2240000
2260000
2280000
2300000
2320000
2340000
2360000
2380000
2400000
2420000
2440000
2460000
2480000
2500000
2520000
2540000
2560000
2580000
2600000
2620000
26400

In [10]:
len(dic.keys())

4326933

In [11]:
count = 0
tweet_url_df = pd.DataFrame(columns=['tweet_id', 'country', 'state', 'url'])
def myFun(row):
    try:
        global count
        
        count=count+1
        if count%1000 == 0:
            print count
        tweet_id = row[0]
        if tweet_id not in dic:
            return 1
        urls = dic[tweet_id]
        for url in  urls:
            #tweet_url_df.loc[tweet_url_df.shape[0]] =
            return {"url":url, "tweet_id":tweet_id, "country":row[2], "state" : row[1]}

    except Exception as e:
        print(e)
        traceback.print_exc()
    return 1
    

pool = ThreadPool(2)
location_url_result = pool.map(myFun, location_tweet_data)

10002000

3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
53000
54000
55000
55000
56000
5700058000

59000
6000060000

61000
62000
63000
64000
65000
66000
67000
6800069000

7000070000

 71000
72000
73000
74000
75000
7600076000

77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
8700088000

89000
90000
91000
92000
93000
94000
95000
96000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151


1155000
1156000
1157000
1158000
11590001160000
1161000
1162000
1163000
1164000
1165000
1166000
1167000
1168000
1169000
1170000

1171000
1172000
1173000
1174000
1175000
 1177000
1178000
11790001176000
1180000

1181000
1182000
1183000
1184000
1185000
1186000
1187000
 1188000
1189000
1190000
11910001192000
1193000
1194000

11950001196000
1197000
1198000
1199000
1200000
1201000

1202000
1203000
1204000
1205000
1206000
1207000
1208000
1209000
1210000
1211000
1212000
1213000
1214000
 1215000
1216000
1217000
1218000
1219000
1220000
1221000
1222000
1223000
1224000
1225000
1226000
1227000
1228000
 1230000
1231000
1232000
1233000
1234000
1235000
12360001229000

12370001238000
1239000
1240000
1241000

1242000
1243000
1244000
1245000
1246000
1247000
 1249000
1250000
1251000
1248000
1252000
1253000
12540001255000

1256000
1257000
1258000
12590001260000
1261000
1262000
1263000
1264000
1265000
1266000
1267000
1268000
1269000

1270000
1271000
1272000
1273000
12740001274000
1275000
1276000
1277000
127

### Get worry data

In [12]:
from micromort.data_stores.mongodb import getConnection
mongo_connection = getConnection("micromort", "newstweets_categorized_news")

In [15]:
count = 0
worry_url_dic = {}
def myFun(row):
    try:
        global count
        global worry_url_dic
        worry_url_dic[row["url"]] = row
        count=count+1
        if count%10000 ==0:
            print count
   
    except Exception as e:
        print(e)
        traceback.print_exc()
    return 1
    

pool = ThreadPool(6)
result = pool.imap(myFun, mongo_connection.find({"$or" : [{"polarity" : "Verynegative"}, {"polarity" : "Negative"}]}, 
                                {"url":1, "original_url":1,"polarity":1,"labels":1}))

In [23]:
mongo_connection.find({"$or" : [{"polarity" : "Verynegative"}, {"polarity" : "Negative"}]}, {"url":1, "original_url":1,"polarity":1,"labels":1}).count()


20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000


251445

### Merge all of them

In [24]:
len(worry_url_dic.keys())

246770

In [25]:
location_url_result[:10]

250000


[1,
 {'country': 'Netherlands',
  'state': '',
  'tweet_id': 995956477167898625L,
  'url': 'https://www.theguardian.com/environment/2018/mar/02/arctic-spring-is-starting-16-days-earlier-than-a-decade-ago-study-shows?CMP=share_btn_tw'},
 1,
 1,
 {'country': 'United Kingdom',
  'state': 'England',
  'tweet_id': 995956478854000641L,
  'url': 'http://www.bbc.com/news/world-asia-44105279'},
 {'country': 'Kenya',
  'state': 'Nairobi',
  'tweet_id': 995956485929734145L,
  'url': 'https://www.nytimes.com/2018/05/13/nyregion/marijuana-arrests-nyc-race.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=second-column-region&region=top-news&WT.nav=top-news'},
 {'country': 'Malaysia',
  'state': 'Selangor',
  'tweet_id': 995956487913549824L,
  'url': 'https://www.thestar.com.my/news/nation/2018/05/12/stop-accepting-barisan-leaders-into-pakatan-dr-m-told/'},
 {'country': 'India',
  'state': 'Madhya Pradesh',
  'tweet_id': 995956488739803136L,
  'url': 'https://www.ndtv.com/busines

In [26]:
len(location_url_result)

2149252

In [27]:
counter = 0
i = 0
tweet_url_df_f =  pd.DataFrame(columns=['tweet_id', 'country', 'state', 'url', 'parsed_url', 'worry', 'polarity'])
def myFun(r):
    try:
        global counter
        global worry_url_dic
        global i
        global tweet_url_df_f
        counter=counter+1
        if counter%10000 == 0:
            print counter
        if r != 1:
            url = r["url"]
            if url not in worry_url_dic:
                return 1
            
            worry_map = worry_url_dic[url]
            for worry in worry_map["labels"]:
             
                return {
                    'tweet_id': r["tweet_id"], 
                    'country' : r["country"], 
                    'state': r["state"], 
                    'url' : url, 
                    'parsed_url' : worry_map["url"], 
                    'worry' : worry, 
                    'polarity' : worry_map["polarity"]
                }
    except Exception as e:
        print(e)
        traceback.print_exc()
    return 1
    

pool = ThreadPool(6)
final_results = pool.map(myFun, location_url_result)

1000020000

30000
40000
50000
6000070000
 80000
90000
 100000

110000
120000
130000130000
140000

150000160000
170000
160000

 170000
 180000
190000
170000200000
200000

 220000220000
210000
 230000

240000
 260000
270000250000
 
280000
290000
300000
310000
320000
320000
330000340000

 350000
360000370000
360000

370000
380000390000400000

410000
 
420000
430000430000
 
440000
 450000
 460000470000480000
490000


500000
510000520000
530000

510000
540000
550000560000
 
 580000570000

580000590000600000


600000610000620000


630000
630000
 630000
640000
650000
650000
660000
660000
670000
670000
 680000680000

670000 
 680000
700000690000

710000720000

 710000710000720000


730000740000

 750000760000
750000

770000
780000780000

790000
800000
810000
 830000
820000
840000
850000
 860000
860000
870000880000

890000
890000
900000
910000
920000
930000
940000
950000
960000970000
960000

 970000
 980000
980000990000
1000000

1010000
 1020000
10300001040000

1050000
1060000
1070000
1070000
1

In [33]:
final_results[:100]

[1,
 1,
 1,
 1,
 {'country': 'United Kingdom',
  'parsed_url': u'http://www.bbc.com/news/world-asia-44105279',
  'polarity': u'Negative',
  'state': 'England',
  'tweet_id': 995956478854000641L,
  'url': 'http://www.bbc.com/news/world-asia-44105279',
  'worry': 92},
 {'country': 'Kenya',
  'parsed_url': u'https://www.nytimes.com/2018/05/13/nyregion/marijuana-arrests-nyc-race.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=second-column-region&region=top-news&WT.nav=top-news',
  'polarity': u'Negative',
  'state': 'Nairobi',
  'tweet_id': 995956485929734145L,
  'url': 'https://www.nytimes.com/2018/05/13/nyregion/marijuana-arrests-nyc-race.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=second-column-region&region=top-news&WT.nav=top-news',
  'worry': 99},
 {'country': 'Malaysia',
  'parsed_url': u'https://www.thestar.com.my/news/nation/2018/05/12/stop-accepting-barisan-leaders-into-pakatan-dr-m-told/',
  'polarity': u'Negative',
  'state': 'Se

### in case you are OOM:

In [29]:
import json
with open(name='../../data/final_results.json', mode='w') as outfile:
    json.dump(final_results, outfile, ensure_ascii=False)

In [2]:
with io.open("../../data/final_results.json", encoding='latin-1') as f:
    final_results = json.load(f)

In [30]:
final_data_fil = []
for row in final_results:
    if row != 1:
        final_data_fil.append(row)

In [31]:
final_data_df = pd.DataFrame(final_data_fil)

In [34]:
final_data_df[final_data_df.worry <> 99].shape

(501106, 7)

In [9]:
worry_distribution = []
for row in pd.DataFrame({'count' : final_data_df.groupby( ["worry"] ).size()}).reset_index().values:
    worry_distribution.append([classes[row[0]], row[1]])

In [8]:
classes = {
                      91: 'health', 92: 'safety_security', 93 : 'environment',
                      94 : 'social_relations', 95 : 'meaning_in_life', 96 : 'achievement',
                      97 : 'economics', 98 : 'politics', 99 : 'not_applicable', 0 : 'skip' }

In [10]:
worry_distribution

[['health', 35585],
 ['safety_security', 132875],
 ['environment', 8352],
 ['social_relations', 66690],
 ['meaning_in_life', 8659],
 ['achievement', 55],
 ['economics', 4382],
 ['politics', 102877],
 ['not_applicable', 601589]]

In [138]:
for row in pd.DataFrame({'count' : final_data_df[final_data_df.worry == 98].groupby( ["country"] ).size()}).reset_index().values:
    if row[0] not in countries_above_the_arr:
        continue
    normailzed = float(row[1] * 100)/ final_data_df[final_data_df.country == row[0]][final_data_df.worry != 99].shape[0]
    print row[0] + "," + str(normailzed)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Australia,36.1620057859
Belgium,21.5563506261
Canada,18.8713048671
Colombia,2.88379983036
France,23.1991525424
Germany,22.0398183723
India,25.8989409736
Indonesia,3.6832412523
Ireland,20.5944055944
Italy,45.322697607
Japan,14.2204454597
Malaysia,12.5156271704
Netherlands,14.7260273973
Norway,19.2931633835
Pakistan,2.85923753666
Philippines,45.8533975388
Singapore,15.4678548695
South Africa,13.1413141314
Spain,29.1512915129
Sweden,13.5345666991
Thailand,14.7132169576
United Arab Emirates,21.4086471409
United Kingdom,26.4764594187
United States,31.5879238472


In [66]:
countries_above_the = final_data_df[final_data_df.worry <> 99].groupby( ["state"] ).size().reset_index()

In [67]:
countries_above_the_arr = []
for row in countries_above_the[countries_above_the[0] > 1000].values:
    countries_above_the_arr.append(row[0])
countries_above_the_arr

['',
 'Alabama',
 'Alberta',
 'Andhra Pradesh',
 'Arizona',
 'Arkansas',
 'British Columbia',
 'California',
 'Catalonia',
 'Central Singapore',
 'Colorado',
 'Connecticut',
 'Delhi',
 'District of Columbia',
 'Dubai',
 'England',
 'Florida',
 'Georgia',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Karnataka',
 'Kentucky',
 'Louisiana',
 'Madhya Pradesh',
 'Maharashtra',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Missouri',
 'National Capital Region',
 'Nevada',
 'New Jersey',
 'New Mexico',
 'New South Wales',
 'New York',
 'North Carolina',
 'Ohio',
 'Oklahoma',
 'Ontario',
 'Oregon',
 'Pennsylvania',
 'Punjab',
 'Queensland',
 'Rajasthan',
 'Scotland',
 'South Carolina',
 'Tamil Nadu',
 'Tennessee',
 'Texas',
 'Tokyo Prefecture',
 'Utah',
 'Victoria',
 'Virginia',
 'Wales',
 'Washington',
 'West Bengal',
 'Wilayah Persekutuan',
 'Wisconsin',
 '\xcele-de-France']

In [139]:
states_above_the = final_data_df[final_data_df.worry <> 99].groupby( ["state"] ).size().reset_index()

In [143]:
states_above_the_arr = states_above_the[states_above_the[0]>1000].state.values

In [145]:
states_above_the_arr[0] = "XXXX"

In [154]:
for row in pd.DataFrame({'count' : final_data_df[final_data_df.worry == 98].groupby( ["state"] ).size()}).reset_index().values:
    if row[0] not in states_above_the_arr:
        continue
    normailzed = float(row[1] * 100)/ final_data_df[final_data_df.state == row[0]][final_data_df.worry != 99].shape[0]
    print row[0] + "," + str(normailzed)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Alabama,37.7533039648
Alberta,18.1474480151
Andhra Pradesh,26.6737232072
Arizona,33.120362737
Arkansas,26.6832917706
British Columbia,19.6942974721
California,30.5989040029
Catalonia,41.5222772277
Central Singapore,15.4974489796
Colorado,30.6766428107
Connecticut,32.6833403985
Delhi,24.4872591672
District of Columbia,30.7130180603
Dubai,21.0930009588
England,24.8075466165
Florida,33.8803912558
Georgia,31.6987740806
Illinois,31.1424903723
Indiana,31.0755148741
Iowa,32.6171875
Kansas,29.9619771863
Karnataka,33.8640043173
Kentucky,30.8705193855
Louisiana,31.9115323855
Madhya Pradesh,25.2810351799
Maharashtra,25.0024774552
Maryland,31.4356435644
Massachusetts,30.053338238
Michigan,32.4284819378
Minnesota,28.3783783784
Missouri,32.309807516
National Capital Region,44.1364605544
Nevada,32.9008341057
New Jersey,31.8104906937
New Mexico,30.7894736842
New South Wales,34.4321031888
New York,31.1508541794
North Carolina,33.287178067
Ohio,31.7932797663
Oklahoma,32.2307039864
Ontario,17.7413544669


In [88]:
len(final_data_df[final_data_df.worry != 99].tweet_id.unique())

359475

In [128]:
final_data_df[final_data_df.worry<>99][final_data_df.country=="India"][final_data_df.worry==93]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,country,parsed_url,polarity,state,tweet_id,url,worry
407,India,http://indianexpress.com/article/opinion/colum...,Negative,Madhya Pradesh,995957828996055040,http://indianexpress.com/article/opinion/colum...,93
770,India,http://indianexpress.com/article/cities/ludhia...,Negative,,995959116244774912,http://indianexpress.com/article/cities/ludhia...,93
2703,India,http://www.thehindu.com/opinion/editorial/in-a...,Negative,Kerala,995966771558367233,http://www.thehindu.com/opinion/editorial/in-a...,93
2841,India,http://indianexpress.com/article/opinion/colum...,Negative,,995967349290426369,http://indianexpress.com/article/opinion/colum...,93
3066,India,http://indianexpress.com/article/opinion/colum...,Negative,West Bengal,995968174456684544,http://indianexpress.com/article/opinion/colum...,93
3160,India,https://www.theguardian.com/environment/climat...,Verynegative,Madhya Pradesh,995968500387627008,https://www.theguardian.com/environment/climat...,93
3185,India,https://www.theguardian.com/environment/2018/m...,Negative,Madhya Pradesh,995968612312563712,https://www.theguardian.com/environment/2018/m...,93
3298,India,https://www.theguardian.com/environment/2018/m...,Negative,Madhya Pradesh,995969036272857088,https://www.theguardian.com/environment/2018/m...,93
3349,India,https://www.theguardian.com/environment/climat...,Negative,Madhya Pradesh,995969158570364928,https://www.theguardian.com/environment/climat...,93
5051,India,http://www.thehindu.com/todays-paper/tp-featur...,Negative,Maharashtra,995974851218235392,http://www.thehindu.com/todays-paper/tp-featur...,93


### Domain name distribution

In [35]:
final_data_df[final_data_df.worry<>99].shape

(501106, 7)

In [61]:
def myfun(url):
    try:
        parsed_uri = urlparse(url)
        return '{uri.netloc}'.format(uri=parsed_uri)
    except Exception as e:
        print e
        return 1 

pool = ThreadPool(6)
parsed_domains = pool.map(myfun, final_data_df.parsed_url.values)

In [62]:
len(list(set(parsed_domains)))

3405

In [63]:
from collections import Counter
c = Counter(parsed_domains)

In [64]:
for row in c.items():
    print row[0], ",", row[1]

www.thebureauinvestigates.com , 3
iheartprinciples.com , 3
wsvn.com , 4
www.lgbtqnation.com , 4
www.thetablet.co.uk , 1
www.patheos.com , 52
www.correodelorinoco.gob.ve , 2
www.runnymedetrust.org , 2
www.trtworld.com , 1
www.fongbeefze.com , 3
www.defendevropa.org , 1
adorans.org , 1
www.asahi.com , 504
www.encontrocursos.com , 1
topglobalnews.org , 22
theguardian.newspapers.com , 1
www.tempo-team.com , 2
iicybersecurity.wordpress.com , 1
www.depo.gal , 1
nationalspeak.in , 30
www.emptywheel.net , 1
yourlogicalfallacyis.com , 1
www.amazon.fr , 3
developer.worldnow.com , 1
www.pluralist.com , 1
www.facebook.com , 1455
capitalandmain.com , 8
www.lifesitenews.com , 26
www.arthousecrouchend.co.uk , 1
www2.le.ac.uk , 2
www.devex.com , 21
cer.eu , 2
www.taz.de , 1
english.enabbaladi.net , 3
www.madamemorbid.com , 2
www.mintpressnews.com , 5
baltimorefishbowl.com , 21
www.spencerfernando.com , 1
www.13abc.com , 4
emcrit.org , 1
justiceupheld.org.uk , 1
www-nst-com-my.cdn.ampproject.org , 2
ku

www.duurzaambedrijfsleven.nl , 1
boulwareenterprises.wordpress.com , 4
metro.co.uk , 1
themetalden.com , 1
kmu.thehindueducationfair.com , 26
rockinst.org , 3
www.lifewatch.be , 1
www.tellychakkar.com , 6
www.rich.co.ke , 2
hedgeclippers.org , 1
www.khinsider.com , 8
www.watershed.co.uk , 1
abundanthope.net , 1
m.dw.com , 1
www.autocar.co.uk , 1
christ.direct , 5
blogs.mediapart.fr , 1
susanorji.blog , 1
www.climatechangenews.com , 1
www.kremlinpost.com , 1
www.uber.com , 3
www.richmix.org.uk , 7
usafacts.org , 3
actionnetwork.org , 14
infoproc.blogspot.com , 1
www.opensecrets.org , 1
punchdrink.com , 6
seattle.curbed.com , 1
www.unitedwaydm.org , 1
kalundethescribe.wordpress.com , 1
www.thescore.com , 1
www.stitcher.com , 2
electronicintifada.net , 7
uk.trendolizer.com , 2
www.edgeprop.sg , 1
www.japantimes.co.jp , 2034
hoffmanncentre.chathamhouse.org , 1
welfaretales.wordpress.com , 1
www.taydrafahie.com , 1
www.citjotv.com , 4
www.ilsole24ore.com , 12
irishlitsoc.org , 2
www.america

www.garoweonline.com , 1
www.arcada.fi , 2
www.scmp.com , 140
www.booksmith.com , 2
www.allkpop.com , 1
www.goodlifedeathgrief.org.uk , 5
www.njspotlight.com , 1
www.crikey.com.au , 1
www.ksn.com , 4
www.altpress.com , 3
lovinmalta.com , 4
freedomrings.news , 1
www.aljazeera.com , 85
act.amnestyusa.org , 5
au.be.yahoo.com , 1
www.modernhealthcare.com , 1
www.chron.com , 1
www.jrf.org.uk , 1
www.stockholmresilience.org , 2
timesevents.nytimes.com , 51
www.livehindustan.com , 1
www.wcpo.com , 2
www.maxim-pr.co.uk , 2
100noticias.com.ni , 3
cobrapost.com , 1
pcinnewsroom.wordpress.com , 1
www.modernwhig.org , 1
johneagleland2.wordpress.com , 3
www.americanprogress.org , 4
independentaustralia.net , 3
www.newsx.com , 1
www.verifiedvoting.org , 1
whyy.org , 1
www.londonguitaracademy.com , 1
newsinfo.inquirer.net , 3
expressindia.indianexpress.com , 3
economix.blogs.nytimes.com , 8
www.stagwellgroup.com , 3
re.ukri.org , 2
100percentfedup.com , 9
www.ncronline.org , 9
www.thepostemail.com , 

www.lemonde.fr , 2
inpursuitofhappiness.wordpress.com , 87
mobile.reuters.com , 2
www.mediamasters.fm , 5
www.bing-amp.com , 1
www.brooklyneagle.com , 7
www.sny.tv , 1
beta.finance.yahoo.com , 2
www.bizpacreview.com , 61
business.mb.com.ph , 133
www.wsj.com , 411
www.powellmediainc.com , 1
thedeepstate.com , 1
joy.org.au , 1
www.nydailynews.com , 195
www.plutobooks.com , 6
newswars.com , 2
www.laprensa.com.ni , 3
bbc.com.cmun.it , 1
m.realestatelistings.nytimes.com , 2
www.rcplondon.ac.uk , 3
www.statista.com , 2
www.channelnonfiction.com , 1
blog.ivyglobal.com , 1
forward.com , 8
www.realclearinvestigations.com , 1
www.bristolpost.co.uk , 5
kq94.net , 8
daiphongland.com , 1
www.opednews.com , 3
thegoldwater.com , 4
www.apostagem.com.br , 2
blog.dopebeat.com.ng , 10
www.dbusiness.com , 1
www.si.com , 1
au.news.yahoo.com , 1692
www.bognor.co.uk , 1
www.local10.com , 1
newrepublic.com , 6
www.conservativehome.com , 3
www.ttweets.com , 2
news.berkeley.edu , 1
www.ranger-core.net , 2
www.c

jacobyandmeyers.com , 1
tvnews4u.com , 1
kottke.org , 2
sja.nujs.edu , 3
www.channelnewsasia.com , 2147
www1.nst.com.my , 3
www.smithsonianmag.com , 1
www.hitc.com , 2
thisislagos.ng , 4
manilastandard.net , 4
www.belfastlive.co.uk , 7
howiecarrshow.com , 29
educate.itsfacile.com , 2
www.cdc.gov , 2
flipboard.com , 1
edition.cnn.com , 7319
www.themorningchronicle.in , 1
www.greenwichfilm.org , 1
www.gmwatch.org , 1
www.brathay.org.uk , 4
m.patrika.com , 2
www.news9.com , 2
www.bapco.org.uk , 4
shop.bbc.com , 8
khabar.ndtv.com , 257
www.axios.com , 7
www.volksverpetzer.de , 3
www.mcall.com , 1
www.heraldo.mx , 1
www.wradio.com.co , 1
dealbook.nytimes.com , 46
www.insideindonesia.org , 4
niemanreports.org , 2
www.tcpalm.com , 11
www.livemint.com , 40
byggindustrin.se , 1
ascopubs.org , 1
www.scielo.org.ar , 1
m.theepochtimes.com , 1
challenger-aaa.tumblr.com , 1
www.tennismagazin.de , 1
www.urbandictionary.com , 1
m.chron.com , 2
trumptrainnews.com , 1
www.greaterkashmir.com , 1
www.kurd